## Filtrage par gemini 1.5 Flash

In [10]:
import os
import pandas as pd
from dotenv import load_dotenv
import google.generativeai as genai
import ast
import time

iteration = 6

# Chargement des variables d'environnement
load_dotenv()

# Configuration de l'API Gemini
genai.configure(api_key=os.getenv("API_KEY"))

# Définition du system_prompt et du prompt utilisateur
system_prompt = '''
You are an expert Smartphone analyzer. Your task is to analyze an image of a smartphone on a white background and answer specific questions about it. Here is the image you need to analyze:
Please carefully examine the image, then answer the following questions.
'''

# Liste des questions (facilement modifiable pour les futurs prompts)
questions = [
    'Is there one smartphone on a white background? (Answer only by "Yes" or "No")',
    'Which face of the Smartphone is shown: the front (screen side) or back (camera side) or other side of the device? (Answer only by "Front" or "Back" or "Other")',
]

# Construction du prompt utilisateur à partir de la liste des questions
prompt = '\n'.join(questions)

# Nom du modèle et configuration de génération
model_name = "gemini-1.5-flash"
generation_config = {
    "temperature": 0.2,
    "top_p": 0.9,
    "top_k": 40,
    "max_output_tokens": 200,
    "response_mime_type": "text/plain",
}

In [11]:
# Fonction pour analyser une image avec l'IA
def analyze_image(image_path, system_prompt, prompt, model_name, generation_config):
    """Analyse une image en utilisant le modèle Gemini et retourne les réponses."""
    try:
        time.sleep(0.01)
        # Télécharger l'image vers Gemini
        file = genai.upload_file(image_path)
        # Créer une session de chat avec le modèle
        model = genai.GenerativeModel(
            model_name=model_name,
            generation_config=generation_config,
            system_instruction=system_prompt,
        )
        chat_session = model.start_chat(history=[])
        # Préparer le message avec l'image et le prompt
        message = [file, prompt]
        # Envoyer le message et obtenir la réponse
        response = chat_session.send_message(message)
        
        print(f'Image: {image_path} correctly analyzed')
        return response.text.strip()
    except Exception as e:
        print(f"Erreur lors de l'analyse de l'image {image_path}: {e}")
        return None

In [12]:
# Chargement du DataFrame principal
df_ai = pd.read_csv(f'Data/{iteration}/smartphone_infos_{iteration}.csv')

# Conversion de la colonne 'Images_Path' de chaîne en liste si nécessaire
df_ai['Images_Path'] = df_ai['Images_Path'].apply(ast.literal_eval)

# Nombre de questions
num_questions = len(questions)

# Initialisation des colonnes pour stocker les réponses
for i in range(1, num_questions + 1):
    df_ai[f'Answer_{i}'] = [[] for _ in range(len(df_ai))]

In [13]:
df_ai

,Unnamed: 0,title,link,price,description,publication_date,Condition,Brand,Model,Color,Memory,images,Spare_Parts_Availability,Refurbished_Item_Condition,Reparability_Index,Images_Path,Answer_1,Answer_2
0,1,IPhone 11,https://www.leboncoin.fr/ad/telephones_objets_...,255 €,iPhone 11 jaune 64go en très bonne état 100% d...,2024-11-16 21:21:00,3,Apple,iPhone 11,Jaune,64 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,NaN,NaN,NaN,[produit_0_image_0.jpg],[],[]
1,4,IPhone 7 💫,https://www.leboncoin.fr/ad/telephones_objets_...,50 €,"Je vends un IPHONE\nJ’ai la GARANTIE, et la FA...",2024-10-08 01:06:00,3,Apple,iPhone 7,Noir,32 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,Non renseignée,NaN,NaN,[produit_1_image_0.jpg],[],[]
2,9,Iphone 15 pro Max,https://www.leboncoin.fr/ad/telephones_objets_...,1 000 €,"Vends iPhone 15 Pro Max couleur titane, très b...",2024-11-16 21:21:00,3,Apple,iPhone 15 Pro Max,Argent / Silver,256 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,NaN,NaN,NaN,[produit_2_image_0.jpg],[],[]
3,10,IPhone apple,https://www.leboncoin.fr/ad/telephones_objets_...,70 €,"iPhone en excellent état,vendu avec garanti et...",2024-11-23 16:42:00,3,Apple,iPhone SE,Rouge,64 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,NaN,NaN,NaN,"[produit_3_image_0.jpg, produit_3_image_1.jpg]",[],[]
4,11,Réveil,https://www.leboncoin.fr/ad/telephones_objets_...,10 €,Vends réveil en très bon état.,2024-11-16 21:22:00,3,NaN,NaN,Noir,8 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,NaN,NaN,NaN,"[produit_4_image_0.jpg, produit_4_image_1.jpg,...",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,2984,Iphone 13 à vendre sur Paris,https://www.leboncoin.fr/ad/telephones_objets_...,350 €,"Vendu avec facture, accessoires d’origine et g...",2024-11-25 18:39:00,4,Apple,iPhone 13,Violet,128 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,Non disponible,Très bon état,NaN,"[produit_1714_image_0.jpg, produit_1714_image_...",[],[]
1715,2985,IPhone 13 128GO blanc,https://www.leboncoin.fr/ad/telephones_objets_...,450 €,"Vend iPhone 13 128GO , en excellent état \nT...",2024-11-25 18:39:00,4,Apple,iPhone 13,Blanc,128 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,NaN,NaN,NaN,"[produit_1715_image_0.jpg, produit_1715_image_...",[],[]
1716,2993,Iphone 13 128Go Blanc,https://www.leboncoin.fr/ad/telephones_objets_...,390 €,Je vends mon iPhone 13 128Go car j'ai acheté u...,2024-11-25 18:41:00,3,Apple,iPhone 13,Blanc,128 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,NaN,NaN,NaN,"[produit_1716_image_0.jpg, produit_1716_image_...",[],[]
1717,2995,Iphone 11,https://www.leboncoin.fr/ad/telephones_objets_...,300 €,"Bonjour, je vend mon iphone 11 pour m’acheter ...",2024-11-25 18:43:00,3,Apple,iPhone 11,Noir,64 Go,['https://img.leboncoin.fr/api/v1/lbcpb1/image...,NaN,NaN,NaN,"[produit_1717_image_0.jpg, produit_1717_image_...",[],[]


In [14]:
# Itération sur chaque produit et ses images
for idx, row in df_ai.iterrows():
    images = row['Images_Path']
    answers_per_question = [[] for _ in range(num_questions)]
    for image_name in images:
        image_path = f"Data/{iteration}/images_without_background/{image_name}"
        # Analyse de l'image avec l'IA
        response = analyze_image(image_path, system_prompt, prompt, model_name, generation_config)
        if response:
            # Traitement de la réponse
            answers = response.strip().split('\n')
            answers = [ans.strip() for ans in answers if ans.strip()]
            # Vérification du nombre de réponses
            if len(answers) != num_questions:
                print(f"Avertissement: {num_questions} réponses attendues mais {len(answers)} reçues pour l'image {image_name}")
                continue
            for i in range(num_questions):
                answers_per_question[i].append(answers[i])
        else:
            for i in range(num_questions):
                answers_per_question[i].append(None)
    # Stockage des réponses pour ce produit
    for i in range(num_questions):
        df_ai.at[idx, f'Answer_{i + 1}'] = answers_per_question[i]



Image: Data/6/images_without_background/produit_0_image_0.jpg correctly analyzed
Image: Data/6/images_without_background/produit_1_image_0.jpg correctly analyzed
Image: Data/6/images_without_background/produit_2_image_0.jpg correctly analyzed
Image: Data/6/images_without_background/produit_3_image_0.jpg correctly analyzed
Image: Data/6/images_without_background/produit_3_image_1.jpg correctly analyzed
Image: Data/6/images_without_background/produit_4_image_0.jpg correctly analyzed
Image: Data/6/images_without_background/produit_4_image_1.jpg correctly analyzed
Image: Data/6/images_without_background/produit_4_image_2.jpg correctly analyzed
Image: Data/6/images_without_background/produit_4_image_3.jpg correctly analyzed
Image: Data/6/images_without_background/produit_5_image_0.jpg correctly analyzed
Image: Data/6/images_without_background/produit_5_image_1.jpg correctly analyzed
Image: Data/6/images_without_background/produit_6_image_0.jpg correctly analyzed
Image: Data/6/images_without

In [15]:
# Filtrage des images en fonction des réponses
df_selected = pd.DataFrame(columns=['Image_Path', 'Condition', *[f'Answer_{i + 1}' for i in range(num_questions)]])

for idx, row in df_ai.iterrows():
    condition = row['Condition']
    images = row['Images_Path']
    answers_list = [row[f'Answer_{i + 1}'] for i in range(num_questions)]
    for image_path, *answers in zip(images, *answers_list):
        # Critères de filtrage (modifiable pour futurs prompts)
        if answers[0] == 'Yes':
            new_row = {
                'Image_Path': image_path,
                'Condition': condition,
                **{f'Answer_{i + 1}': answers[i] for i in range(num_questions)}
            }
            df_selected = pd.concat([df_selected, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Image {image_path} ne répond pas aux critères")

# Séparation des images en fonction de la face du smartphone
df_front = df_selected[df_selected['Answer_2'] == 'Front'][['Image_Path', 'Condition']]
df_back = df_selected[df_selected['Answer_2'] == 'Back'][['Image_Path', 'Condition']]

# Sauvegarder les DataFrames traités
df_front.to_csv(f'Data/{iteration}/df_front_{iteration}.csv', index=False)
df_back.to_csv(f'Data/{iteration}/df_back_{iteration}.csv', index=False)

df_front

Image produit_1_image_0.jpg ne répond pas aux critères
Image produit_4_image_0.jpg ne répond pas aux critères
Image produit_4_image_1.jpg ne répond pas aux critères
Image produit_4_image_2.jpg ne répond pas aux critères
Image produit_4_image_3.jpg ne répond pas aux critères
Image produit_8_image_4.jpg ne répond pas aux critères
Image produit_13_image_0.jpg ne répond pas aux critères
Image produit_13_image_3.jpg ne répond pas aux critères
Image produit_13_image_4.jpg ne répond pas aux critères
Image produit_20_image_3.jpg ne répond pas aux critères
Image produit_21_image_2.jpg ne répond pas aux critères
Image produit_27_image_3.jpg ne répond pas aux critères
Image produit_33_image_3.jpg ne répond pas aux critères
Image produit_37_image_0.jpg ne répond pas aux critères
Image produit_37_image_1.jpg ne répond pas aux critères
Image produit_37_image_2.jpg ne répond pas aux critères
Image produit_37_image_3.jpg ne répond pas aux critères
Image produit_37_image_4.jpg ne répond pas aux critère

,Image_Path,Condition
0,produit_0_image_0.jpg,3
3,produit_3_image_1.jpg,3
4,produit_5_image_0.jpg,2
7,produit_6_image_1.jpg,4
9,produit_7_image_0.jpg,2
...,...,...
4782,produit_1717_image_0.jpg,3
4783,produit_1717_image_1.jpg,3
4787,produit_1718_image_0.jpg,3
4788,produit_1718_image_1.jpg,3


# Non utilisé pour l'instant

## Évaluation de la pertinence

In [16]:
"""
# Fonction pour générer le prompt dynamique
def dynamic_prompt(condition):
    condition_mapping = {
        0: 'For parts',
        1: 'Satisfactory condition',
        2: 'Good condition',
        3: 'Very good condition',
        4: 'Brand new',
    }
    state = condition_mapping.get(condition, 'Unknown condition')
    
    prompt = f'''
    The claimed condition for this smartphone is: {state}

    Please analyze the image carefully to determine if the claimed condition approximately matches the actual visual condition.

    Based on your analysis, respond accordingly.
    '''
    return prompt


# Définition du system_prompt
system_prompt = '''
You are an expert in smartphone evaluation. Your task is to compare the claimed condition of a smartphone with its actual visual condition based on the provided image.

Possible conditions are:
- 'For parts'
- 'Satisfactory condition'
- 'Good condition'
- 'Very good condition'
- 'Brand new'

Instructions:
- If the claimed condition approximately matches the visual condition of the smartphone in the image, respond with "Okay".
- If it does not match, provide the condition level that is one level higher or lower than the claimed condition, as appropriate.
- Respond only with "Okay" or the adjusted condition level. Do not include any explanations or additional information.
'''

"""


'\n# Fonction pour générer le prompt dynamique\ndef dynamic_prompt(condition):\n    condition_mapping = {\n        0: \'For parts\',\n        1: \'Satisfactory condition\',\n        2: \'Good condition\',\n        3: \'Very good condition\',\n        4: \'Brand new\',\n    }\n    state = condition_mapping.get(condition, \'Unknown condition\')\n    \n    prompt = f\'\'\'\n    The claimed condition for this smartphone is: {state}\n\n    Please analyze the image carefully to determine if the claimed condition approximately matches the actual visual condition.\n\n    Based on your analysis, respond accordingly.\n    \'\'\'\n    return prompt\n\n\n# Définition du system_prompt\nsystem_prompt = \'\'\'\nYou are an expert in smartphone evaluation. Your task is to compare the claimed condition of a smartphone with its actual visual condition based on the provided image.\n\nPossible conditions are:\n- \'For parts\'\n- \'Satisfactory condition\'\n- \'Good condition\'\n- \'Very good condition\'\n-

In [17]:
"""

# Fonction pour mapper l'état à un numéro
def state_number(condition, state):
    state_mapping = {
        'For parts': 0,
        'Satisfactory condition': 1,
        'Good condition': 2,
        'Very good condition': 3,
        'Brand new': 4,
        'Okay': condition,
    }
    return state_mapping.get(state, None)

# Fonction pour traiter le DataFrame
def process_dataframe(model_name, system_prompt, generation_config, dataframe):
    df = dataframe.copy()
    df['Gemini_Response'] = ''

    for idx, row in df.iterrows():
        image_name = row['Image_Path']
        image_path = f"Data/{iteration}/images_without_background/{image_name}"
        
        condition = row['Condition']
        
        # Obtenir la réponse
        answer = analyze_image(
            image_path=image_path, 
            system_prompt=system_prompt, 
            prompt=dynamic_prompt(condition),
            model_name=model_name, 
            generation_config=generation_config
        )

        number = state_number(condition, answer)
        
        # Stocker la réponse
        df.at[idx, 'Gemini_Response'] = number
    return df
    
    
    """

'\n\n# Fonction pour mapper l\'état à un numéro\ndef state_number(condition, state):\n    state_mapping = {\n        \'For parts\': 0,\n        \'Satisfactory condition\': 1,\n        \'Good condition\': 2,\n        \'Very good condition\': 3,\n        \'Brand new\': 4,\n        \'Okay\': condition,\n    }\n    return state_mapping.get(state, None)\n\n# Fonction pour traiter le DataFrame\ndef process_dataframe(model_name, system_prompt, generation_config, dataframe):\n    df = dataframe.copy()\n    df[\'Gemini_Response\'] = \'\'\n\n    for idx, row in df.iterrows():\n        image_name = row[\'Image_Path\']\n        image_path = f"Data/{iteration}/images_without_background/{image_name}"\n        \n        condition = row[\'Condition\']\n        \n        # Obtenir la réponse\n        answer = analyze_image(\n            image_path=image_path, \n            system_prompt=system_prompt, \n            prompt=dynamic_prompt(condition),\n            model_name=model_name, \n            gene

In [18]:
"""

# Nom du modèle et configuration de génération
model_name = "gemini-1.5-flash"
generation_config = {
    "temperature": 0.2,
    "top_p": 0.9,
    "top_k": 40,
    "max_output_tokens": 150,
    "response_mime_type": "text/plain",
}

# Traitement des DataFrames df_front et df_back
df_front_final = process_dataframe(model_name, system_prompt, generation_config, df_front)
df_back_final = process_dataframe(model_name, system_prompt, generation_config, df_back)

# Sauvegarder les DataFrames traités
df_front_final.to_csv(f'Data/{iteration}/df_front_{iteration}.csv', index=False)
df_back_final.to_csv(f'Data/{iteration}/df_back_{iteration}.csv', index=False)

"""

'\n\n# Nom du modèle et configuration de génération\nmodel_name = "gemini-1.5-flash"\ngeneration_config = {\n    "temperature": 0.2,\n    "top_p": 0.9,\n    "top_k": 40,\n    "max_output_tokens": 150,\n    "response_mime_type": "text/plain",\n}\n\n# Traitement des DataFrames df_front et df_back\ndf_front_final = process_dataframe(model_name, system_prompt, generation_config, df_front)\ndf_back_final = process_dataframe(model_name, system_prompt, generation_config, df_back)\n\n# Sauvegarder les DataFrames traités\ndf_front_final.to_csv(f\'Data/{iteration}/df_front_{iteration}.csv\', index=False)\ndf_back_final.to_csv(f\'Data/{iteration}/df_back_{iteration}.csv\', index=False)\n\n'